# Image Content Extraction Using Generative AI  
 This notebook is designed to be user-friendly, even for those with minimal technical background.  
   
## Instructions:  
   
1. **Install Libraries and Dependencies:**  
   - Before running the cells, make sure that all required libraries are installed. This might involve installing packages via pip or conda. Typically, you would use a command like `pip install -r requirements.txt` in your terminal, where `requirements.txt` is a file containing a list of items to be installed.  
   
2. **Run Each Cell:**  
   - Start from the top and run each cell in order by clicking on the cell and pressing the "Run" button or using the shortcut `Shift + Enter`. Make sure to run them sequentially as some cells depend on the output of the previous cells.  
   
3. **Enter Required Information:**  
   - If the notebook requires any inputs (like image URLs or project IDs), make sure to enter them when prompted.  
   
4. **View Results:**  
   - After running the cells related to the AI model invocation, you should see the output below those cells. This output will provide insights into the content of the image based on the AI's analysis.  
   
Feel free to modify the inputs or experiment with different images to see how the model's responses change. Enjoy exploring the capabilities of generative AI in visual content understanding!  
```

In [8]:

from dotenv import load_dotenv
import os
load_dotenv(".env", override=True)
import pandas as pd

from langchain_core.pydantic_v1 import BaseModel, Field
from google.cloud import bigquery
import tqdm

from PIL import Image  
from io import BytesIO  
from IPython.display import Markdown, Video, display

from google.cloud import storage
from PIL import Image  # Using PIL for image manipulation


from pydantic import BaseModel, Field, constr, field_validator  

# Pandas display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)  


from google import genai
from google.genai import types
from pydantic import BaseModel, Field, constr, field_validator  
from typing import List, Union
from typing_extensions import Literal
import json 

import pprint

In [ ]:
# Constants
client = bigquery.Client(os.getenv("PROJECT_ID"))

In [3]:
def image_to_pillow(gcs_url):

    image = Image.open(BytesIO(image_to_bytes(gcs_url)))
    return image

def image_to_bytes(gcs_url):
    """Loads an image from Google Cloud Storage and returns a PIL Image object.
    Args:
        gcs_url (str): Full GCS URL of the image (e.g., "gs://bucket/image.jpg").
    Returns:
        PIL.Image.Image: PIL Image object, or None if an error occurred.
    """
    try:
        bucket_name, blob_name = gcs_url.replace("gs://", "").split("/")[-2:]
        client = storage.Client(os.getenv("PROJECT_ID"))
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        image_bytes = blob.download_as_bytes()
        return image_bytes
    except Exception as e:
        print(f"Error loading image from GCS: {e}")
        return None


In [ ]:
genai_client = genai.Client(
    vertexai=True, project=os.getenv("PROJECT_ID"), location='us-central1'
)

IMAGE_LINK="https://storage.googleapis.com/ds-genai-test-renault-ad-lib-image/ea6e9608-e4bd-4a75-bb6d-9d983ced68f8.jpg"

image_to_pillow(IMAGE_LINK)


In [ ]:
# Bastien --> mettre en place un code de test pour le prompting
bytes = image_to_bytes(IMAGE_LINK)

class ImageInformationTest(BaseModel):  
    """Information about an image."""  

    promotion_display: bool = Field(description="""
                Indicates whether a currency is displayed in the ad. True if 'yes', False if 'no'.
                exemples: 
                    - 100%: False
                    - 100€: True             
            """) 
    call_to_action_verb: str = Field(default="None", description="The specific verb used in the call to action, if present (e.g., 'buy', 'test').")  
    promotion_wording_type: Literal["None", "gain", "loss_aversion"] = Field(default="None", description="Specifies the type of promotion wording used in the ad: 'gain' for emphasizing benefits, 'loss_aversion' for emphasizing avoidance of loss, or 'None' if there is no promotion wording.")  

response = genai_client.models.generate_content(
    model='gemini-2.0-flash-exp', 
    contents=[
        'Describe the content of the picture',
        types.Part.from_bytes(data=bytes, mime_type="image/png")    
    ],
    config=types.GenerateContentConfig(
        temperature=0,
        response_mime_type= 'application/json',
        response_schema= ImageInformationTest
    )
)

feature_dict = json.loads(response.text)
print("="*10,"response", "="*10)
pprint.pprint(feature_dict)